# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [1]:
# all import statements needed for the project, for example:

import math

import bs4
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import requests
import sqlalchemy as db
import re
import os
import glob

In [2]:
# any constants you might need, for example:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
TAXI_ZONES = "taxi_zones/taxi_zones.shp"
# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "uber/uber_rides_sample.csv"
WEATHER_PATH = "weather"


NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library.
* [ ] Taxi data:
    * [ ] Use the `re` module, and the packages `requests`, BeautifulSoup (`bs4`), and (optionally) `pandas` to programmatically download the required CSV files & load into memory.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* [ ] Weather & Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
* [ ] All data:
    * [ ] Load the data using `pandas`
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * (Taxi & Uber data) Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] (Taxi data) Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
* [ ] Weather data:
    * [ ] Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
    * [ ] You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 

### Calculating distance
_**TODO:** Write some prose that tells the reader what you're about to do here._

In [3]:
def calculate_distance(from_coord, to_coord):
    # convert longitude and latitude to radian
    lon1, lat1, lon2, lat2 = map(math.radians, from_coord+to_coord) 
    dlon=lon2-lon1
    dlat=lat2-lat1
    a=math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    distance=2*math.asin(math.sqrt(a))*6371*1000
    distance=round(distance/1000,3)
    return distance

In [4]:
def add_distance_column(dataframe):
    dataframe["distance"] = dataframe.apply(lambda x: calculate_distance((
        x['pickup_longitude'], x['pickup_latitude']), (x['dropoff_longitude'], x['dropoff_latitude'])), axis=1)
    return dataframe


### Processing Taxi Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [5]:
def find_taxi_parquet_urls(taxi_url):
    response = requests.get(taxi_url)
    html = response.content
    results_page = bs4.BeautifulSoup(html, 'html.parser')
    links = results_page.find_all('a',title=r'Yellow Taxi Trip Records')
    urls = [link.get('href') for link in links]
    return urls

In [6]:
def get_month_taxi_data(taxi_data_url):
    pattern = r'yellow_tripdata_[0-9]{4}-[0-9]{2}'
    name = re.search(pattern,taxi_data_url).group()
    path = os.path.join(os.getcwd(),'taxi',name+'.csv')
    if os.path.exists(path):
        print("read")
        df = pd.read_csv(path)
    else:
        print("download",str(taxi_data_url))
        df = pd.read_parquet(taxi_data_url, engine='pyarrow')
        df.to_csv(path)
    return df

In [7]:
def clean_month_taxi_data(taxi_data,taxi_zones):
    taxi_data = taxi_data[["tpep_pickup_datetime", "PULocationID", "DOLocationID","trip_distance"]]
    taxi_data = taxi_data.rename(columns={"tpep_pickup_datetime": "pickup_datetime", "trip_distance": "distance"})
    taxi_data = taxi_data.dropna()
    taxi_data = taxi_data.merge(taxi_zones[["LocationID","lat","lon"]].set_index('LocationID'),
                                left_on="PULocationID", right_on="LocationID")
    taxi_data = taxi_data.rename(columns={"lat": "pickup_latitude", "lon": "pickup_longitude"})
    taxi_data = taxi_data.merge(taxi_zones[["LocationID","lat","lon"]].set_index('LocationID'),
                                left_on="DOLocationID", right_on="LocationID")
    taxi_data = taxi_data.rename(columns={"lat": "dropoff_latitude", "lon": "dropoff_longitude"})
    
    taxi_data = taxi_data.drop(columns=["PULocationID","DOLocationID"])
    
    #pick up in bounding box 
    taxi_data = taxi_data[(taxi_data['pickup_longitude']>NEW_YORK_BOX_COORDS[0][1])
                          & (taxi_data['pickup_longitude']<NEW_YORK_BOX_COORDS[1][1])]
    taxi_data = taxi_data[(taxi_data['pickup_latitude']>NEW_YORK_BOX_COORDS[0][0])
                          & (taxi_data['pickup_latitude']<NEW_YORK_BOX_COORDS[1][0])]
    #drop off in bounding box  
    taxi_data = taxi_data[(taxi_data['dropoff_longitude']>NEW_YORK_BOX_COORDS[0][1])
                          & (taxi_data['dropoff_longitude']<NEW_YORK_BOX_COORDS[1][1])]
    taxi_data = taxi_data[(taxi_data['dropoff_latitude']>NEW_YORK_BOX_COORDS[0][0])
                          & (taxi_data['dropoff_latitude']<NEW_YORK_BOX_COORDS[1][0])]

    return taxi_data

In [8]:
def get_and_clean_taxi_data():
    all_csv_urls = find_taxi_parquet_urls(TAXI_URL)
    taxi_zone_df = gpd.read_file(TAXI_ZONES)
    taxi_zone_df = taxi_zone_df.to_crs(4326)
    taxi_zone_df['lon'] = taxi_zone_df.centroid.x  
    taxi_zone_df['lat'] = taxi_zone_df.centroid.y
    all_taxi_dataframes = []
    for csv_url in all_csv_urls[3:]:
        month_taxi_dataframe = get_month_taxi_data(csv_url)
        month_taxi_dataframe = clean_month_taxi_data(month_taxi_dataframe,taxi_zone_df)
        
        all_taxi_dataframes.append(month_taxi_dataframe.sample(23800,random_state=1))
        
    # create one gigantic dataframe with data from every month needed
    all_taxi_data = pd.concat(all_taxi_dataframes)
    return all_taxi_data

In [ ]:
taxi_data = get_and_clean_taxi_data()
taxi_data

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_9556\346742037.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taxi_zone_df['lon'] = taxi_zone_df.centroid.x
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_9556\346742037.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taxi_zone_df['lat'] = taxi_zone_df.centroid.y


download https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-04.parquet
download https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-05.parquet


### Processing Uber Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
def clean_uber_data(csv_file):
    uber_data = pd.read_csv(csv_file)
    uber_data = uber_data.drop(columns=["Unnamed: 0","key","fare_amount","passenger_count"])
    uber_data = uber_data.dropna()
    
    #pick up in bounding box 
    uber_data = uber_data[(uber_data['pickup_longitude']>NEW_YORK_BOX_COORDS[0][1]) & (uber_data['pickup_longitude']<NEW_YORK_BOX_COORDS[1][1])]
    uber_data = uber_data[(uber_data['pickup_latitude']>NEW_YORK_BOX_COORDS[0][0]) & (uber_data['pickup_latitude']<NEW_YORK_BOX_COORDS[1][0])]
    #drop off in bounding box  
    uber_data = uber_data[(uber_data['dropoff_longitude']>NEW_YORK_BOX_COORDS[0][1]) & (uber_data['dropoff_longitude']<NEW_YORK_BOX_COORDS[1][1])]
    uber_data = uber_data[(uber_data['dropoff_latitude']>NEW_YORK_BOX_COORDS[0][0]) & (uber_data['dropoff_latitude']<NEW_YORK_BOX_COORDS[1][0])]

    return uber_data

In [ ]:
def get_uber_data():
    uber_dataframe = clean_uber_data(UBER_CSV)
    uber_dataframe = add_distance_column(uber_dataframe)
    return uber_dataframe

In [ ]:
uber_data = get_uber_data()
uber_data

### Processing Weather Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
def clean_month_weather_data_hourly(csv_file):
    hourly_data = pd.read_csv(csv_file)
    print(hourly_data.info())
    hourly_data = hourly_data[hourly_data['REPORT_TYPE'] != 'SOD']
    hourly_data = hourly_data[['DATE','HourlyPresentWeatherType','HourlyPrecipitation','HourlyWindSpeed']]
    hourly_data['DATE'] = pd.to_datetime(hourly_data['DATE'])
    hourly_data['HourlyPrecipitation'] = pd.to_numeric(hourly_data['HourlyPrecipitation'], errors='coerce')
    hourly_data['HourlyWindSpeed'] = pd.to_numeric(hourly_data['HourlyWindSpeed'], errors='coerce')
    hourly_data['HourlyPrecipitation'].fillna(0, inplace=True)
    hourly_data = hourly_data.dropna()
    return hourly_data

In [ ]:
def clean_month_weather_data_daily(csv_file):
    daily_data = pd.read_csv(csv_file)
    daily_data = daily_data[daily_data['REPORT_TYPE'] == 'SOD']
    daily_data = daily_data[['DATE','Sunrise','Sunset']]
    daily_data = daily_data.dropna()
    return daily_data

In [ ]:
def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []
    
    weather_csv_files = glob.glob(os.path.join(os.getcwd(), WEATHER_PATH, "*.csv"))
    
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

In [ ]:
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()
hourly_weather_data

In [ ]:
daily_weather_data

### Process All Data

_This is where you can actually execute all the required functions._

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
taxi_data = get_and_clean_taxi_data()
uber_data = get_uber_data()
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

## Part 2: Storing Cleaned Data

_Write some prose that tells the reader what you're about to do here._

In [ ]:
engine = db.create_engine(DATABASE_URL)

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather
(
    id INTEGER PRIMARY KEY,
    datetime DATETIME,
    HourlyWindSpeed FLOAT32,
    HourlyPrecipitation FLOAT32
);
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather
(
    id INTEGER PRIMARY KEY,
    DATE DATE,
    Sunrise INT32,
    Sunset INT32
);
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips
(
    id INTEGER PRIMARY KEY,
    pickup_datetime DATETIME,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT,
    passenger_count INTEGER,
    distance FLOAT,
    fare_amount FLOAT
);
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips
(
    id INTEGER PRIMARY KEY,
    pickup_datetime DATETIME,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT,
    passenger_count INTEGER,
    distance FLOAT,
    fare_amount FLOAT
);
"""

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [ ]:
# create the tables with the schema files
with engine.connect() as connection:
    sql = open(DATABASE_SCHEMA_FILE, "r")
    commands = sql.read().split(";")
    sql.close()
    for command in commands:
        connection.execute(command)

### Add Data to Database

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
def write_dataframes_to_table(table_to_df_dict):
    for name, table in table_to_df_dict.items():
        table.to_sql(name, engine, if_exists='replace', index=False)

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [ ]:
def write_query_to_file(query, outfile):
    with open(outfile, "w") as f:
        f.write(query)

### Query N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each query_

For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.

In [ ]:
QUERY_1 = """
SELECT strftime('%H', pickup_datetime) AS time, COUNT(*) AS num
FROM taxi_trips
GROUP BY time
ORDER BY num DESC;
"""

In [ ]:
engine.execute(QUERY_1).fetchall()

In [ ]:
write_query_to_file(QUERY_1, "most_popular_hour.sql")

In [ ]:
QUERY_2 = """
SELECT strftime('%w', pickup_datetime) AS day, COUNT(*) AS num
FROM uber_trips
GROUP BY day
ORDER BY num DESC;
"""

In [ ]:
engine.execute(QUERY_2).fetchall()

In [ ]:
write_query_to_file(QUERY_2, ".sql")

In [ ]:
QUERY_3 = """
SELECT strftime('%H', pickup_datetime) AS time, COUNT(*) AS num
FROM taxi_trips
GROUP BY time
ORDER BY num DESC;
"""

In [ ]:
engine.execute(QUERY_3).fetchall()

In [ ]:
write_query_to_file(QUERY_3, "most_popular_day.sql")

In [ ]:
QUERY_4 = """
WITH hired_trips AS (SELECT pickup_datetime,distance 
      FROM taxi_trips 
      WHERE pickup_datetime BETWEEN '2013-07-01' AND '2013-08-01'
      UNION ALL
      SELECT pickup_datetime,distance FROM uber_trips
      WHERE pickup_datetime BETWEEN '2013-07-01' AND '2013-08-01')
SELECT distance
FROM hired_trips
ORDER BY distance ASC
LIMIT 1
OFFSET (SELECT COUNT(*) FROM hired_trips) * 95 / 100 - 1 ;
"""

In [ ]:
engine.execute(QUERY_4).fetchall()

In [ ]:
write_query_to_file(QUERY_4, "95%_percentile_distance.sql")

In [ ]:
QUERY_5 = """
SELECT date(pickup_datetime) AS date, COUNT(*) AS num, 
FROM (SELECT pickup_datetime FROM taxi_trips
      UNION ALL
      SELECT pickup_datetime FROM uber_trips)
GROUP BY date
HAVING date IN (SELECT date(DATE) FROM daily_weather WHERE DATE BETWEEN '2014-01-01' AND '2015-01-01' ORDER BY DailyWindSpeed DESC LIMIT 10)
"""

In [ ]:
engine.execute(QUERY_5).fetchall()

In [ ]:
write_query_to_file(QUERY_5, "top_10_windiest_days.sql")

In [ ]:
QUERY_6 = """
WITH hired_trips AS (SELECT strftime('%Y-%m-%d %H:00:00:00',pickup_datetime) AS DATE
      FROM taxi_trips
      WHERE pickup_datetime BETWEEN '2012-10-22' AND '2012-10-31'
      UNION ALL
      SELECT pickup_datetime FROM uber_trips
      WHERE pickup_datetime BETWEEN '2012-10-22' AND '2012-10-31')
SELECT strftime('%Y-%m-%d %H:00:00:00',hourly_weather.DATE) AS WDATE, COALESCE(COUNT(hired_trips.DATE),0) AS num, HourlyPrecipitation, HourlyWindSpeed
FROM hourly_weather
LEFT JOIN hired_trips
ON WDATE = hired_trips.DATE
WHERE WDATE BETWEEN '2012-10-22' AND '2012-10-31'
GROUP BY WDATE
"""

In [ ]:
engine.execute(QUERY_6).fetchall()

In [ ]:
write_query_to_file(QUERY_6, "hurricane_sandy.sql")

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [ ]:
# use a more descriptive name for your function
def plot_visual_n(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_n():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplemented()

In [ ]:
some_dataframe = get_data_for_visual_n()
plot_visual_n(some_dataframe)